# 기본 세팅

In [1]:
import numpy as np 
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('fivethirtyeight')
plt.ion()

import warnings
warnings.filterwarnings('ignore')

In [6]:
import easydict
args = easydict.EasyDict() #dict values 에 .으로 접근 가능하다 

# path 정보
args.default_path = 'C:\\Users\\User\\Desktop\\' # 파일 위치 
args.train_csv = args.default_path+'train.csv'
args.test_csv = args.default_path+'test.csv'
args.default_submission_csv = args.default_path+'submission_0330.csv'

args.submission_csv = args.default_path+'submission_0330.csv' # 새로운 파일 
args.save_results = args.default_path+"model_results.csv" # 정보저장 파일 

# 데이터 분석을 위한 변수들
args.random_state = 21
args.results = []

args.save_results

'C:\\Users\\User\\Desktop\\model_results.csv'

# 이전 파일 부르기

In [ ]:
ori_train = pd.read_csv(args.train_csv)
ori_test = pd.read_csv(args.test_csv)
ori_train.shape, ori_test.shape

In [13]:
pre__result = pd.read_csv(args.save_results)
pre__result

,Unnamed: 0,model,score_tr,score_te,auc_te,len_features,feaute_importances,create_dt
0,1,modelV1,0.982839,0.781818,0.773645,10,"['gender_female', 'fare', 'age', 'pclass', 'si...",217
1,0,modelV0,0.982839,0.778182,0.768612,10,"['gender_female', 'fare', 'age', 'pclass', 'si...",217
2,2,modelV2,0.982839,0.774545,0.763580,10,"['gender_female', 'fare', 'age', 'pclass', 'si...",217


# *Base ModelV1*

In [ ]:
train = X_tr.copy() 
test = X_te.copy()
ori_te = ori_test.copy()

train.shape, test.shape, ori_te.shape

((641, 10), (275, 10), (393, 10))

### Data preprocessing

In [ ]:
print(f'before: {train.shape} / {test.shape}')
drop_cols = ['name', 'ticket', 'cabin']

train.drop(drop_cols, axis=1, inplace=True)
test.drop(drop_cols, axis=1, inplace=True)
ori_te.drop(drop_cols, axis=1, inplace=True)


print(f'after: {train.shape} / {test.shape}')

before: (641, 10) / (275, 10)
after: (641, 7) / (275, 7)


#### 결측치 처리

In [ ]:
age_median = train['age'].median()
fare_median = train['fare'].median()
embarked_mode = train['embarked'].mode().values[0]

age_median, fare_median, embarked_mode

(28.0, 14.4, 'S')

In [ ]:
train['age'].fillna(age_median, inplace=True)
test['age'].fillna(age_median, inplace=True)
ori_te['age'].fillna(age_median, inplace=True)

train['fare'].fillna(fare_median, inplace=True)
test['fare'].fillna(fare_median, inplace=True)
ori_te['fare'].fillna(fare_median, inplace=True)

train['embarked'].fillna(embarked_mode, inplace=True)
test['embarked'].fillna(embarked_mode, inplace=True)
ori_te['embarked'].fillna(embarked_mode, inplace=True)

train.isnull().sum().sum(), test.isnull().sum().sum(), ori_te.isnull().sum().sum()

(0, 0, 0)

### data encoding

In [ ]:
enc_cols = ['gender', 'embarked']
normal_cols = list(set(train.columns) - set(enc_cols))
normal_cols

['pclass', 'sibsp', 'parch', 'age', 'fare']

In [ ]:
enc = OneHotEncoder()

tmp_tr = pd.DataFrame(
    enc.fit_transform(train[enc_cols]).toarray(), #범주형 encoder 적용시켜서 temp_tr에 저장
    columns = enc.get_feature_names_out() # feature name을 뽑아냄 
)  
enc_tr = pd.concat(
    [train[normal_cols].reset_index(drop=True), tmp_tr.reset_index(drop=True)]
    , axis=1
) 
# 수치형 데이터(train[normal_cols]) + encoding 된 문자형 데이터(tmp_tr) 를 합쳐서 enc_tr



# test (ori_te 에서 나온 te  -> 모델의 학습을 평가(잘했는지 못했는지))
tmp_te = pd.DataFrame(
    enc.transform(test[enc_cols]).toarray(), 
    columns = enc.get_feature_names_out()
)
enc_te = pd.concat(
    [test[normal_cols].reset_index(drop=True), tmp_te.reset_index(drop=True)]
    , axis=1
)


# ori_test
tmp_te = pd.DataFrame(
    enc.transform(ori_te[enc_cols]).toarray(), 
    columns = enc.get_feature_names_out()
)
enc_ori_te = pd.concat(
    [ori_te[normal_cols].reset_index(drop=True), tmp_te.reset_index(drop=True)]
    , axis=1
)
print(f'before: {train.shape} / {test.shape} / {ori_te.shape}')
print(f'after: {enc_tr.shape} / {enc_te.shape} / {enc_ori_te.shape}')
# gender가 두개로 , embarked가 3개로 

before: (641, 7) / (275, 7) / (393, 7)
after: (641, 10) / (275, 10) / (393, 10)


### Scaler

In [ ]:
enc_tr.columns

Index(['pclass', 'sibsp', 'parch', 'age', 'fare', 'gender_female',
       'gender_male', 'embarked_C', 'embarked_Q', 'embarked_S'],
      dtype='object')

In [ ]:
scaling_cols = ['age', 'fare']
not_scaling_cols = list(set(enc_tr.columns) - set(scaling_cols))
not_scaling_cols 

['pclass',
 'sibsp',
 'gender_male',
 'parch',
 'gender_female',
 'embarked_Q',
 'embarked_S',
 'embarked_C']

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
std = StandardScaler()

_scaled_tr = std.fit_transform(enc_tr[scaling_cols])
_scaled_te = std.transform(enc_te[scaling_cols])
_scaled_ori_te = std.transform(enc_ori_te[scaling_cols])

In [ ]:
print(f'before: {enc_tr.shape} / {enc_te.shape} / {enc_ori_te.shape}')
# train
tmp_tr = pd.DataFrame(
    _scaled_tr, 
    columns = scaling_cols
)
scaled_tr = pd.concat(
    [enc_tr[not_scaling_cols].reset_index(drop=True), tmp_tr.reset_index(drop=True)]
    , axis=1
).reset_index(drop=True)

# test
tmp_te = pd.DataFrame(
    _scaled_te, 
    columns = scaling_cols
)
scaled_te = pd.concat(
    [enc_te[not_scaling_cols].reset_index(drop=True), tmp_te.reset_index(drop=True)]
    , axis=1
).reset_index(drop=True)

# ori_test
tmp_te = pd.DataFrame(
    _scaled_ori_te, 
    columns = scaling_cols
)
scaled_ori_te = pd.concat(
    [enc_ori_te[not_scaling_cols].reset_index(drop=True), tmp_te.reset_index(drop=True)]
    , axis=1
).reset_index(drop=True)


print(f'after: {scaled_tr.shape} / {scaled_te.shape} / {scaled_ori_te.shape}')

before: (641, 10) / (275, 10) / (393, 10)
after: (641, 10) / (275, 10) / (393, 10)


### Training

In [ ]:
modelV1 = DecisionTreeClassifier(random_state=21)

modelV1.fit(scaled_tr, y_tr)

DecisionTreeClassifier(random_state=21)

### Evaluation

In [ ]:
score_tr = modelV1.score(scaled_tr, y_tr)
score_te = modelV1.score(scaled_te, y_te) 

score_tr, score_te  

(0.982839313572543, 0.7818181818181819)

In [ ]:
from sklearn.metrics import roc_curve, auc 

y_pred = modelV1.predict_proba(scaled_te)[:,1]
fpr, tpr, thresholds = roc_curve(y_te,y_pred)
auc_te = auc(fpr, tpr)
print(f'model: {auc_te}')

model: 0.7736448493027441


In [ ]:
ori_te_pred = modelV1.predict_proba(scaled_ori_te)[:,1]

In [ ]:
df_feature_importances = pd.DataFrame(modelV1.feature_importances_, scaled_tr.columns).sort_values(by=[0], ascending=False).reset_index()
print(f'{df_feature_importances.shape}')

(10, 2)


In [ ]:
args.results.append(
    {
        'model': 'modelV1',
        'score_tr': score_tr,
        'score_te': score_te,
        'auc_te': auc_te,
        'ori_te_pred': ori_te_pred,
        'len_features': X_tr.shape[1],
        'feaute_importances': list(df_feature_importances['index'].values[:X_tr.shape[1]]),
        'create_dt': '0330'
    }
)